In [41]:
import praw
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import defaultdict

# Initialize Reddit API client
reddit = praw.Reddit(
    client_id="lMYhh7BNAjLrJQ_c7v0M5w",
    client_secret="Mq0R3nUw3FxCmfnqcLM4sIrLA1_dzg",
    user_agent="your_user_agent"
)

# Initialize VADER sentiment analyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Define subreddits
fitness_subreddits = ["Fitness", "bodybuilding", "running", "sports", "weightlifting"]
art_subreddits = ["ArtistLounge", "ArtCrit"]

# Initialize sets to store user data
fitness_users = set()
art_users = set()

# Initialize dictionaries for sentiment scores
sentiment_scores_both = defaultdict(list)
sentiment_scores_art_only = defaultdict(list)

# Helper function to extract users from a subreddit
def extract_users(subreddit_name, user_set, limit=100):
    try:
        subreddit = reddit.subreddit(subreddit_name)
        for submission in subreddit.new(limit=limit):
            if submission.author:
                user_set.add(submission.author.name)
    except Exception as e:
        print(f"Error extracting from {subreddit_name}: {e}")

# Helper function to measure sentiment of posts
def analyze_sentiment(usernames, group_name, limit=10):
    sentiment_scores = defaultdict(list)
    for username in usernames:
        try:
            user = reddit.redditor(username)
            for submission in user.submissions.new(limit=limit):
                sentiment = sia.polarity_scores(submission.title + " " + (submission.selftext or ""))
                sentiment_scores[group_name].append(sentiment["compound"])
        except Exception as e:
            print(f"Error analyzing sentiment for {username}: {e}")
    return sentiment_scores

# Extract users from fitness/sport-related subreddits
for sub in fitness_subreddits:
    print(f"Extracting users from {sub}...")
    extract_users(sub, fitness_users)

# Extract users from art-related subreddits
for sub in art_subreddits:
    print(f"Extracting users from {sub}...")
    extract_users(sub, art_users)

# Find users who post in both categories
both_categories_users = art_users.intersection(fitness_users)

# Find users who post only in art subreddits
art_only_users = art_users.difference(fitness_users)

# Analyze sentiment for both groups
print("Analyzing sentiment for users in both fitness and art subreddits...")
sentiment_scores_both = analyze_sentiment(both_categories_users, "both")

print("Analyzing sentiment for users only in art subreddits...")
sentiment_scores_art_only = analyze_sentiment(art_only_users, "art_only")

# Calculate average sentiment scores
def average_sentiment(scores):
    if scores:
        return sum(scores) / len(scores)
    return 0

avg_sentiment_both = average_sentiment(sentiment_scores_both["both"])
avg_sentiment_art_only = average_sentiment(sentiment_scores_art_only["art_only"])

# Print results
print(f"Average sentiment for users in both categories: {avg_sentiment_both:.2f}")
print(f"Average sentiment for users only in art subreddits: {avg_sentiment_art_only:.2f}")

# Save results to files
with open("sentiment_scores_both.txt", "w") as f:
    for score in sentiment_scores_both["both"]:
        f.write(f"{score}\n")

with open("sentiment_scores_art_only.txt", "w") as f:
    for score in sentiment_scores_art_only["art_only"]:
        f.write(f"{score}\n")

print("Sentiment analysis complete!")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/dsi/saadaty/nltk_data...


Extracting users from Fitness...
Extracting users from bodybuilding...
Extracting users from running...
Extracting users from sports...
Extracting users from weightlifting...
Extracting users from ArtistLounge...
Extracting users from ArtCrit...
Analyzing sentiment for users in both fitness and art subreddits...
Analyzing sentiment for users only in art subreddits...
Average sentiment for users in both categories: 0.00
Average sentiment for users only in art subreddits: 0.29
Sentiment analysis complete!


In [40]:
!pip install praw nltk


     |████████████████████████████████| 1.5 MB 760 kB/s eta 0:00:01
